In [24]:
print("Ok")

Ok


In [25]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\Logiks_solutions_chatbot'

In [32]:
import os
os.chdir("Logiks_solutions_chatbot/logiks_solutions_chatbot")

In [33]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\Logiks_solutions_chatbot\\logiks_solutions_chatbot'

In [137]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [139]:
# Extract data from the PDF file
def load_pdf_file(data):
    loader= DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [140]:
extracted_data = load_pdf_file(data='Data/')

In [ ]:
#extracted_data

In [ ]:
# Split the data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [142]:
text_chunks = text_split(extracted_data)

print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  19


In [ ]:
#text_chunks

In [143]:
from langchain.embeddings import HuggingFaceEmbeddings

In [145]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') #384 dimension
    
    return embeddings

In [146]:
embeddings = download_hugging_face_embeddings()

In [147]:
query_result = embeddings.embed_query("Hello World")
print("Length: ", len(query_result))

Length:  384


In [ ]:
#query_result

In [148]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [149]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [164]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "companychatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "companychatbot",
    "metric": "cosine",
    "host": "companychatbot-zmnjrwu.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [150]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [165]:
# Embed each chunk and save the embeddings into Pinecone database
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [166]:
# Load existing index   

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [167]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={"k": 3})

In [168]:
retrieved_docs = retriever.invoke("What is Jobstell?")

In [169]:
retrieved_docs

[Document(id='337e52bd-7221-4732-9eb1-97a4034b666c', metadata={'author': 'aleksi modebadze', 'creationdate': '2025-02-12T00:15:48+04:00', 'creator': 'Microsoft® Word 2021', 'moddate': '2025-02-12T00:15:48+04:00', 'page': 6.0, 'page_label': '7', 'producer': 'Microsoft® Word 2021', 'source': 'Data\\Jobstell_AI_Knowledge.pdf', 'total_pages': 8.0}, page_content='10. Is my data secure on Jobstell? \nYes, Jobstell takes user privacy and security seriously. All data, including voice recordings and \nresponses, is stored securely. Jobstell does not share user data with third parties, and users can \nrequest data deletion by contacting support. \n11. What languages does Jobstell support? \nCurrently, Jobstell supports English. Future plans include expanding language support, but as of \nnow, the platform is focused on English language learners.'),
 Document(id='abeba6b3-f1ec-4349-98e5-a3b454ef7775', metadata={'author': 'aleksi modebadze', 'creationdate': '2025-02-12T00:15:48+04:00', 'creator': 

In [171]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1",
    temperature=0.4,
    max_tokens=500
)

In [172]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks about the company. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [173]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [174]:
response = rag_chain.invoke({"input": "who is hitler?"})
print(response["answer"])

The provided context does not contain information about Hitler. If you need information about Jobstell or its services, please let me know.
